In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
from html import unescape

In [2]:
tlkCont = pd.read_pickle('../datasets/rawVideosContent.pkl')
print(len(tlkCont))
tlkCont.head(2)

96131


,videoId,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,publishDate
qNvKWhzdpW4,qNvKWhzdpW4,0,0,1001,"[TEDxTalks, English, Life, Anthropology, Compl...",222,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Dr. Kyle William Bishop is a Cedar City native...,1.57697e+09,2019-05-15 00:00:00
huOxk-UhOz0,huOxk-UhOz0,468,4,1161,"[TEDxTalks, English, United States, Life, Fail...",26243,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","With an honest and vulnerable approach, Joey H...",1.57697e+09,2016-05-17 00:00:00


In [3]:
tlkBasic = pd.read_pickle('../datasets/rawVideos.pkl')
print(len(tlkBasic))
tlkBasic.head(2)

96728


,videoId,titleRaw,shortDesc,publishedTimeRaw,viewCountRaw,lengthRaw
-eLgCDFKD7s,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,Nikit talks about wildlife and his work within...,7 months ago,546 views,16:46
nqSXwWUwpRM,nqSXwWUwpRM,"Puppets, guardian angels of the culture | Pou...",Persian puppets have a reach history and backg...,7 months ago,407 views,18:58


In [6]:
tlkCont.loc['nWecIwtN2ho']

videoId                                                   nWecIwtN2ho
likes                                                          101249
dislikes                                                         4204
lengthSeconds                                                    1204
keywords            [Andorra, Spanish, TEDx, TEDxAndorralaVella, t...
views                                                         7768520
talkText            <?xml version="1.0" encoding="utf-8" ?><transc...
videoDescription    Never miss a talk! SUBSCRIBE to the TEDx chann...
currentDate                                               1.57697e+09
publishDate                                       2013-10-12 00:00:00
Name: nWecIwtN2ho, dtype: object

In [4]:
def extractData(raw, regExp, cols=['title', 'speaker', 'event']):
    missing = raw[pd.isna(raw.event)].index.values
    titleEvent2 = raw.loc[missing, :].titleRaw.str.extract(regExp)
    raw.loc[missing, cols[0]] = titleEvent2.loc[:, 0]
    raw.loc[missing, cols[1]] = titleEvent2.loc[:, 1]
    raw.loc[missing, cols[2]] = titleEvent2.loc[:, 2]
    return raw

def cleanRaw(raw):
    raw.insert(0, 'title', np.nan)
    raw.insert(1, 'speaker', np.nan)
    raw.insert(2, 'event', np.nan)
    raw = extractData(raw, r'(.+) \| ([^|]+) \| ([^|]+)$')
    raw = extractData(raw, r'(.+) │ (.+) │ (.+)')
    raw = extractData(raw, r'(.+): (.+) at (\w+)')
    raw = extractData(raw, r'(.+) - (.+) - (.+)')
    raw = extractData(raw, r'(.+) - (.+) at (.+)')
    raw = extractData(raw, r'()(TEDx.+) - (.+)', ['title', 'event', 'speaker'])
    raw = extractData(raw, r'()(.+) at (TEDx.+)')
    return raw[(~pd.isna(raw.title)) & (~pd.isna(raw.talkText))]

In [5]:
talks = tlkBasic.merge(tlkCont, how='inner', on='videoId')
talks.index = talks.videoId.values
talks = cleanRaw(talks.copy())
talks.drop(['viewCountRaw', 'lengthRaw', 'shortDesc'], axis=1, inplace=True)
print(len(talks), 'charlas')
talks.head(2)

78391 charlas


,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,publishDate
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,7 months ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Nikit talks about wildlife and his work within...,1.57697e+09,2019-05-17 00:00:00
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,7 months ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,2019-05-17 00:00:00


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

In [7]:
from multiprocessing import  Pool
from functools import partial

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [8]:
# unescape(' '.join([text.text for text in BeautifulSoup(talks.sample(1).talkText[0], 'lxml').findAll('text')])).replace('\n', ' ')

In [17]:
def decodeTalk(text):
    return BeautifulSoup(unescape(BeautifulSoup(text, 'lxml').text), 'lxml').text

texts = talks.talkText.apply(decodeTalk)

In [18]:
talks.loc[:,'text'] =  texts

In [20]:
talks.drop(['talkText'], axis=1, inplace=True)
talks.head(2)

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,videoDescription,currentDate,publishDate,text
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,7 months ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,Nikit talks about wildlife and his work within...,1.57697e+09,2019-05-17 00:00:00,I don't have never was in Mumbai so muchall of...
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,7 months ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,2019-05-17 00:00:00,I always loved the Mediterranean maybe because...


In [25]:
talks['lengthSeconds'] = talks.lengthSeconds.astype(int)
talks['views'] = talks.views.astype(int)
talks['likes'] = talks.likes.astype(int)
talks['dislikes'] = talks.dislikes.astype(int)

In [33]:
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')
from nltk.tokenize import sent_tokenize, word_tokenize

def only_nouns(texts):
    output = []
    NOUN_CODES = ['NN', 'NNP', 'NNS', 'NNPS']
    for doc in tqdm(texts):
        noun_text = ' '.join([cat[0] for cat in nltk.pos_tag(word_tokenize(doc)) if cat[1] in NOUN_CODES]).lower()
        output.append(noun_text)
    return output


# only_nouns(talksSample.text[:20])

talks['nouns'] = only_nouns(talks.text)

100%|██████████████████████████████████████████████████████████████████████████| 78391/78391 [1:19:25<00:00, 16.45it/s]


In [32]:
only_nouns(['I am here to talk about climate change'])

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.12it/s]


['climate change']

In [34]:
talks

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,videoDescription,currentDate,publishDate,text,nouns
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,7 months ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,Nikit talks about wildlife and his work within...,1.57697e+09,2019-05-17 00:00:00,I don't have never was in Mumbai so muchall of...,mumbai muchall cause three-second ofthe pinto ...
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,7 months ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,2019-05-17 00:00:00,I always loved the Mediterranean maybe because...,mediterranean north phrase feride scenography ...
587FCkioktM,Ce que j’ai appris de 15 000km à vélo,Pierre Bouillot,TEDxINSA,587FCkioktM,Ce que j’ai appris de 15 000km à vélo | Pierre...,7 months ago,6,0,890,"[TEDxTalks, French, Life, Adventure, Explorati...",283,"Diplômé de l'INSA Lyon en 2016, Pierre Bouillo...",1.57699e+09,2019-05-17 00:00:00,we're fine great started you go open your eye...,eyes images breakup years today share bike tri...
UY5bDA63b7k,O Poder do Imprevisto na Música,André Santos,TEDxFunchal,UY5bDA63b7k,O Poder do Imprevisto na Música | André Santos...,7 months ago,44,1,1032,"[TEDxTalks, Portuguese, Art, Connection, Initi...",1164,"André Santos é um guitarrista madeirense, que ...",1.57699e+09,2019-05-17 00:00:00,[Applause] we are ready for the court on the...,[ applause court day invitation today talk mar...
xUZf2ZUl1xA,My mom read the news every day. How journalism...,Florence Rogers,TEDxValleyHS,xUZf2ZUl1xA,My mom read the news every day. How journalism...,1 week ago,3,0,880,"[TEDxTalks, English, Humanities, Humanity, Ide...",108,My mom read the news every day. How journalism...,1.57699e+09,2019-05-17 00:00:00,[Applause]so I just came back from the Isle of...,[ applause ] isle island southcoast englandmy ...
m5jeaTARaUE,The Only Way To Predict The Future Is To Creat...,Bethany Girma,TEDxValleyHS,m5jeaTARaUE,The Only Way To Predict The Future Is To Creat...,1 week ago,23,8,514,"[TEDxTalks, English, Life, Achievement, Advent...",413,The Only Way To Predict The Future Is To Creat...,1.57699e+09,2019-05-17 00:00:00,[Applause]good morning ladies and gentlemen my...,[ applause ] morning ladies gentlemen myname b...
12D6gTp2924,Come e perchè mantenere pulito lo Spazio,Rosario Nasca,TEDxBarletta,12D6gTp2924,Come e perchè mantenere pulito lo Spazio | Ros...,7 months ago,20,0,851,"[TEDxTalks, Italian, Global Issues, Environmen...",373,Un viaggio nello spazio alla ricerca dell'impr...,1.57699e+09,2019-05-17 00:00:00,[Music] [Applause] imagine yourself sitting ...,music ] [ applause ] sky starry summer night s...
P50Rk_WozbU,Banking for the Unbanked - Katharine Budd,Katharine Budd,TEDxAlWaslWomen,P50Rk_WozbU,Banking for the Unbanked - Katharine Budd | Ka...,7 months ago,3,0,923,"[TEDxTalks, English, Global Issues, Developing...",171,"Katharine is co-founder of Now Money, a platfo...",1.57697e+09,2019-05-17 00:00:00,good morning everybody I'm here today totalk t...,morning everybody today services abit subject ...
mKbbC9zx8oA,Turning Loss into Legacy - Danielle Wilson Naqvi,Danielle Wilson Naqvi,TEDxAlWaslWomen,mKbbC9zx8oA,Turning Loss into Legacy - Danielle Wilson Naq...,7 months ago,0,0,1121,"[TEDxTalks, English, Life, Children, Death, De...",137,"In this talk, Danielle Wilson Naqvi will be sh...",1.57699e+09,2019-05-17 00:00:00,my story of what-ifs is going to take uson a j...,story what-ifs journey years journey inever jo...
Dg5ifmBfBbg,Gender equality: taking action at a local level,Derek O'Connell,TEDxMackay,Dg5ifmBfBbg,Gender equality: taking action at a local leve...,7 months ago,3,1,995,"[TEDxTalks, English, Life, Activism, Evolution...",

In [35]:
talks.to_pickle('../datasets/videosData.pkl')